# Exploratory Data Analysis

# 1. Imports

## 1.1 Packages

In [4]:
import os

import pandas as pd


## 1.2 Options

In [5]:
path_data = '../data/03_primary'

## 1.3 Data

In [6]:
df_train = pd.read_csv(os.path.join(path_data, 'train.csv'))

# 2. Analyze

In [8]:
df_train.sample(2)

,id,WORLDCLIM_BIO1_annual_mean_temperature,WORLDCLIM_BIO12_annual_precipitation,WORLDCLIM_BIO13.BIO14_delta_precipitation_of_wettest_and_dryest_month,WORLDCLIM_BIO15_precipitation_seasonality,WORLDCLIM_BIO4_temperature_seasonality,WORLDCLIM_BIO7_temperature_annual_range,SOIL_bdod_0.5cm_mean_0.01_deg,SOIL_bdod_100.200cm_mean_0.01_deg,SOIL_bdod_15.30cm_mean_0.01_deg,...,X18_mean,X26_mean,X50_mean,X3112_mean,X4_sd,X11_sd,X18_sd,X26_sd,X50_sd,X3112_sd
118,106117608,25.844238,3060.505127,383.326538,50.720558,40.879761,10.322960,100,124,110,...,15.977965,31.903881,1.576368,24518.421496,0.155025,0.874413,0.473036,2.425678,0.091434,1256.151515
5430,180324213,2.232837,885.014282,86.647621,42.860466,642.088684,24.111429,97,145,119,...,0.099267,0.832366,1.259975,459.455830,0.001151,0.289343,0.046607,0.158272,0.023153,69.097672


## 2.1 Prepare

In [ ]:
df_train['file_path'] = df_train['id'].apply(lambda s: f'{path_data}/train_images/{s}.jpeg')